# Python访问接口

接口指应用程序接口（API, Application Programming Interface）
- 广义上，一般是由操作系统或程序库提供给应用程序调用的代码。其主要目的是让应用程序开发人员得以调用一组功能，而无须考虑其底层的代码实现或理解其内部工作机制。包括远程过程调用（RPC）、数据库查询（SQL）、文件传输和数据传输等。
- 狭义上接口指开发上为实现系统各部分相互调用进行的设计和编程。良好的接口设计可以降低系统各部分的相互依赖，实现高内聚、低耦合目标，提高系统的可维护性和扩展性。

可以说对Python标准库和第三方包的调用，都是对接口的调用，标准库和许多的第三方包都是基于Python接口编程的优秀范例。

## 一、Python接口实现和调用

### 1. Python接口定义
定义一个股票行情处理接口。接口的定义主要包含对数据和方法的约定。
- 数据约定，输入输出相关的数据类型和约定
- 方法约定，方法名，参数名，参数和返回值类型，参数顺序等约定

In [1]:
from abc import abstractmethod, ABCMeta
from typing import List, Optional, NoReturn
import logging
import easyquotation
import tushare

logger = logging.getLogger(__name__)

class NoQuotationException(Exception):
    """
    无法获取行情异常
    """
    pass


class SourceType:
    """
    行情源枚举
    """
    EasyQuotation = 1
    TuShare = 2
    

class Quotation:
    """
    行情数据
    """
    def __init__(self):
        self.stock_code: str = ''  # 股票代码
        self.stock_name: str = ''  # 股票名称
        self.trade_date: int = 0  # 交易日期 20190501
        self.trade_time: str = ''  # 交易时间 2019-05-01 10:00:00
        self.price: float = 0.0  # 当前价
        self.open: float = 0.0  # 今日开盘价
        self.pre_close: float = 0.0  # 昨日收盘价

    def __str__(self):
        return '%s %s %f' % (self.trade_time, self.stock_code, self.open)


class StockQuotationInterface(metaclass=ABCMeta):
    """
    行情接口
    """
    @abstractmethod
    def init(self, source=SourceType.EasyQuotation) -> NoReturn:
        """
        初始化
        """
        pass

    @abstractmethod
    def get_realtime_quotations(self, stock_codes: List[str]) -> List[Quotation]:
        """
        获取实时行情
        :param stock_codes: 股票代码
        :return: 行情结果
        """
        pass

### 2. Python接口实现
分别基于EasyQuotation和TuShare实现股票行情处理接口

In [2]:
class EasyQuotationHelper(StockQuotationInterface):
    """
    利用EasyQuotation库实现接口
    """

    def __init__(self, provider='sina', prefix=False):
        """
        初始化
        :param provider: 数据源
        :param prefix: 是否显示股票前缀
        """
        self.provider = provider
        self.prefix = prefix

    def init(self, source=SourceType.EasyQuotation) -> NoReturn:
        pass

    def get_realtime_quotations(self, stock_codes: List[str]) -> List[Quotation]:
        if not stock_codes:
            return []
        handler = easyquotation.use(self.provider)
        results = handler.stocks(stock_codes, prefix=self.prefix)
        if not results:
            raise NoQuotationException('Empty results')
        logger.debug('Get realtime quotations successfully')
        quotations = []
        for key, value in results.items():
            quotation = Quotation()
            quotation.stock_code = key
            quotation.stock_name = value['name']
            quotation.trade_date = int(value['date'].replace('-', ''))
            quotation.trade_time = value['date'] + ' ' + value['time']
            quotation.price = value['now']
            quotation.open = value['open']
            quotation.pre_close = value['close']
            quotations.append(quotation)
        return quotations

In [3]:
class TuShareHelper(StockQuotationInterface):
    """
    利用Tushare库实现接口
    """
    def __init__(self, ts_token=''):
        self.ts_token = ts_token
        self.ts = tushare
        self.ts_pro = None

    def init(self, source=SourceType.EasyQuotation) -> NoReturn:
        if self.ts_token:
            self.ts_pro = self.ts.pro_api(self.ts_token)

    def get_realtime_quotations(self, stock_codes: List[str]) -> List[Quotation]:
        if not stock_codes:
            return []
        results = self.ts.get_realtime_quotes(stock_codes)
        if results is None or results.empty:
            raise NoQuotationException('Empty results')
        logger.debug('Get realtime quotations successfully')
        quotations = []
        for value in results.to_dict('index').values():
            quotation = Quotation()
            quotation.stock_code = value['code']
            quotation.stock_name = value['name']
            quotation.trade_date = int(value['date'].replace('-', ''))
            quotation.trade_time = value['date'] + ' ' + value['time']
            quotation.price = float(value['price'])
            quotation.open = float(value['open'])
            quotation.pre_close = float(value['pre_close'])
            quotations.append(quotation)
        return quotations

### 3. Python基于接口实现代理设计模式

In [8]:
class StockQuotationHelper(StockQuotationInterface):
    """
    代理模式，实现不同源选择
    """
    def __init__(self):
        self.helper: Optional[StockQuotationInterface] = None

    def init(self, source=SourceType.EasyQuotation) -> NoReturn:
        if source == SourceType.EasyQuotation:
            self.helper = EasyQuotationHelper()
            logger.info('Use easyquotation library')
        elif source == SourceType.TuShare:
            self.helper = TuShareHelper()
            logger.info('Use tushare library')
        if self.helper:
            self.helper.init()

    def get_realtime_quotations(self, stock_codes: List[str]) -> List[Quotation]:
        return self.helper.get_realtime_quotations(stock_codes)

### 4. Python接口调用

In [5]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(thread)d %(levelname)s %(module)s - %(message)s')
helper = StockQuotationHelper()
helper.init(source=SourceType.TuShare)
data = helper.get_realtime_quotations(['600120'])
for i in data:
    print(i)

2021-06-08 09:33:43,669 4436127232 INFO <ipython-input-4-aaac73d1bb87> - Use tushare library
2021-06-08 09:33:43,786 4436127232 DEBUG <ipython-input-3-e8c29f931e85> - Get realtime quotations successfully


2021-06-08 09:33:41 600120 5.830000


## 二、基于HTTP的接口

由于网络服务越来越复杂，各种客户端和服务端之间需要一种统一机制进行数据交互。基于HTTP协议的应用程序接口开始出现并逐渐形成标准，如RESTful API。

### 1. 基于HTTP协议从新浪获取股票行情数据，实现股票行情处理接口

In [6]:
import requests
import time
import re

class SinaQuotationHelper(StockQuotationInterface):

    def __init__(self):
        self.headers = {
            "Accept-Encoding": "gzip, deflate, sdch",
            "User-Agent": (
                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                "(KHTML, like Gecko) Chrome/54.0.2840.100 "
                "Safari/537.36"
            ),
        }
        self.grep_detail = re.compile(r"(\d+)=[^\s]([^\s,]+?)%s%s" % (r",([\.\d]+)" * 29, r",([-\.\d:]+)" * 2))

    @staticmethod
    def get_exchange_prefix(stock_code: str) -> str:
        """判断股票ID对应的证券市场
        :param stock_code:股票ID, 若以 'sz', 'sh' 开头直接返回对应类型，否则使用内置规则判断
        :return 'sh' or 'sz'
        """
        sh_head = ("50", "51", "60", "90", "110", "113",
                   "132", "204", "5", "6", "9", "7")
        if stock_code.startswith(("sh", "sz", "zz")):
            return stock_code[:2]
        else:
            return "sh" if stock_code.startswith(sh_head) else "sz"

    @property
    def get_api(self) -> str:
        return f"http://hq.sinajs.cn/rn={int(time.time() * 1000)}&list="

    def init(self, source=SourceType.EasyQuotation) -> NoReturn:
        pass

    def get_realtime_quotations(self, stock_codes: List[str]) -> List[Quotation]:
        if not stock_codes:
            return []
        stock_codes = [self.get_exchange_prefix(stock_code) + stock_code for stock_code in stock_codes]
        params = ','.join(stock_codes)
        response = requests.get(self.get_api + params, headers=self.headers)
        if not response.text:
            raise NoQuotationException('Empty results')
        logger.debug('Get realtime quotations successfully')
        quotations = []
        results = self.grep_detail.finditer(response.text)
        for result in results:
            value = result.groups()
            quotation = Quotation()
            quotation.stock_code = value[0]
            quotation.stock_name = value[1]
            quotation.trade_date = int(value[31].replace('-', ''))
            quotation.trade_time = value[31] + ' ' + value[32]
            quotation.price = float(value[4])
            quotation.open = float(value[2])
            quotation.pre_close = float(value[3])
            quotations.append(quotation)
        return quotations

In [7]:
helper = SinaQuotationHelper()
data = helper.get_realtime_quotations(['600120'])
for i in data:
    print(i)

2021-06-08 16:18:55,740 4436127232 DEBUG connectionpool - Starting new HTTP connection (1): hq.sinajs.cn:80
2021-06-08 16:18:55,809 4436127232 DEBUG connectionpool - http://hq.sinajs.cn:80 "GET /rn=1623140335717&list=sh600120 HTTP/1.1" 200 170
2021-06-08 16:18:55,815 4436127232 DEBUG <ipython-input-6-838d19ff55b0> - Get realtime quotations successfully


2021-06-08 15:00:00 600120 5.830000


### 2. 扩展代理实现类

In [9]:
class SourceType:
    """
    行情源枚举
    """
    EasyQuotation = 1
    TuShare = 2
    Sina = 3

class StockQuotationExHelper(StockQuotationHelper):

    def __init__(self):
        super().__init__()

    def init(self, source=SourceType.EasyQuotation) -> NoReturn:
        if source == SourceType.Sina:
            self.helper = SinaQuotationHelper()
            logger.info('Use sina http api')
        super().init(source=source)

In [10]:
helper = StockQuotationExHelper()
helper.init(source=SourceType.Sina)
data = helper.get_realtime_quotations(['600120'])
for i in data:
    print(i)

2021-06-08 16:28:49,814 4436127232 INFO <ipython-input-9-840bce9897cf> - Use sina http api
2021-06-08 16:28:49,838 4436127232 DEBUG connectionpool - Starting new HTTP connection (1): hq.sinajs.cn:80
2021-06-08 16:28:49,897 4436127232 DEBUG connectionpool - http://hq.sinajs.cn:80 "GET /rn=1623140929832&list=sh600120 HTTP/1.1" 200 170
2021-06-08 16:28:49,902 4436127232 DEBUG <ipython-input-6-838d19ff55b0> - Get realtime quotations successfully


2021-06-08 15:00:00 600120 5.830000


## 三、编程实践：开发一个脚本获取股票实时行情

可以利用第三方包或HTTP API获取，其中可选HTTP行情地址有：
- http://hq.sinajs.cn/rn=1000&list=sh600120,sz000932
- http://qt.gtimg.cn/q=sh600120,sz000932


## 四、期末编程实践：通过Python编程获取和解析互联网中的数据
- 采用面向对象编程，建议利用继承和多态特性
- 采用接口约定，建议对所有输入输出数据和方法都进行约定
- 获取数据量需超过100条，建议格式化输出或存储数据
- 代码格式整洁，结构完整，逻辑清晰，结果合理，建议利用更多已学知识

完成课件中的编程实践和期末编程实践题，结果文件数尽量少，打包在一个压缩包中，发送至qingspace@xmu.edu.cn